In [ ]:
# скачивание и распаковка проекта, распаковка данных
"""!wget https://github.com/gimaevra94/emotions_detector/archive/refs/heads/main.zip
!unzip /content/main.zip
!unzip /content/emotions_detector-main/data.zip"""

#!pip install livelossplot

import tensorflow as tf
import pathlib
import livelossplot

# пути до папок с данными
#path='content/'
path='C:/Users/gimaevra94/Documents/local/emotions_detector-main/data'

# создание каталогов под классы
base_dir=pathlib.Path(path).parent/'data'
train_dir=base_dir/'train'
test_dir=base_dir/'test'

anger_dir=train_dir/'anger'
fear_dir=train_dir/'fear'
happy_dir=train_dir/'happy'
neutral_dir=train_dir/'neutral'
sad_dir=train_dir/'sad'
surprise_dir=train_dir/'surprise'

anger_dir=test_dir/'anger'
fear_dir=test_dir/'fear'
happy_dir=test_dir/'happy'
neutral_dir=test_dir/'neutral'
sad_dir=test_dir/'sad'
surprise_dir=train_dir/'surprise'

# генератор для train
train_gen=tf.keras.preprocessing.image.ImageDataGenerator(rotation_range=40,
                                                          fill_mode='nearest',
                                                          horizontal_flip=True,
                                                          rescale=1./255,
                                                          width_shift_range=0.2,
                                                          height_shift_range=0.2,
                                                          zoom_range=0.2)

# генератор для test
test_gen=tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

# изображения становятся одноканальными 80х80 с сохранением пропорций. перемешивается только train
train=train_gen.flow_from_directory(target_size=(80,80),
                                   color_mode='grayscale',
                                   class_mode='sparse',
                                   directory=train_dir)

test=test_gen.flow_from_directory(target_size=(80,80),
                                   color_mode='grayscale',
                                   class_mode='sparse',
                                   directory=test_dir,
                                    shuffle=False)

# модель сети
model=tf.keras.Sequential([
    tf.keras.layers.Input(shape=(80,80,1)),
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Conv2D(filters=8,kernel_size=3,padding='same',activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=2,strides=None,padding='same'),
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Conv2D(16,3,padding='same',activation='relu'),
    tf.keras.layers.MaxPooling2D(2,None,'same'),
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Conv2D(32,3,padding='same',activation='relu'),
    tf.keras.layers.MaxPooling2D(2,None,'same'),
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Conv2D(64,3,padding='same',activation='relu'),
    tf.keras.layers.MaxPooling2D(2,None,'same'),
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Conv2D(128,3,padding='same',activation='relu'),
    tf.keras.layers.MaxPooling2D(2,None,'same'),
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Conv2D(256,3,padding='same',activation='relu'),
    tf.keras.layers.MaxPooling2D(2,None,'same'),
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(6,'softmax')])

# добавление ошибки, оптимизатора и метрики
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    metrics='accuracy')

# на val_accuracy >=59% коллбек останавливает обучение и сохраняет модель
class MyCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self,epoch,logs=None):
        val_accuracy=logs["val_accuracy"]
        if val_accuracy>=0.5900:
            self.model.stop_training=True
            self.model.save('model.h5')

# обучение
history=model.fit(x=train,
                    epochs=100,
                    validation_data=test,
                    batch_size=64,
                    callbacks=[livelossplot.PlotLossesKeras(),MyCallback()])